In [ ]:

import time

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

from selenium.webdriver.chrome.options import Options

chrome_options = Options()

chrome_options.add_argument('--no-sandbox')  # 解决DevToolsActivePort文件不存在的报错
chrome_options.add_argument('window-size=1920x3000')  # 指定浏览器分辨率
chrome_options.add_argument('--disable-gpu')  # 谷歌文档提到需要加上这个属性来规避bug
chrome_options.add_argument('--hide-scrollbars')  # 隐藏滚动条, 应对一些特殊页面
chrome_options.add_argument(
    'blink-settings=imagesEnabled=false')  # 不加载图片, 提升速度
# 浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
chrome_options.add_argument('--headless')

In [ ]:
# 设置用户名，密码
system_dict = {
    "URL": "https://jcb.buaa.edu.cn/",
    "userid": "XXX",  # 替换为你的用户名
    "password": "XXX",  # 替换为你的密码
}

# 邮箱配置
email_dict = {
    "email": "XXX@qq.com",  # 替换为你的邮箱
    "password": "XXX",  # 替换为你的邮箱密码，对于 QQ 邮箱需要使用授权码
    "reciver": "XXX@qq.com",  # 替换为接收邮件的邮箱列表
    "smtp_server": "smtp.qq.com",  # 替换为你的SMTP服务器地址
    "smtp_port": 587,  # SMTP端口号
}

In [ ]:

# 打开现有窗口
driver = webdriver.Chrome(chrome_options)

# 打开登录页面，url为要打开的地址
driver.get(system_dict["URL"])

# 等待1s
time.sleep(1)

# 找到 a 的 title 为 网上财务系统 的链接
iwc = driver.find_element(By.LINK_TEXT, "网上财务系统")


# 点击链接跳转页面
iwc.send_keys(Keys.ENTER)


# 等待1s
time.sleep(1)

# 跳转页面
driver.switch_to.window(driver.window_handles[-1])
print(driver.current_url)

# 等待1s
time.sleep(1)

In [ ]:

# 元素定位用户名输入框，找到 iframe 里边的 <input type="text" placeholder="请输入学工号" i18n="login.form.schoolid.placeholder" id="unPassword" name="username">

iframe = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe)
username = driver.find_element(By.ID, "unPassword")

# 输入用户名
username.send_keys(system_dict["userid"])

# 元素定位密码输入框
password = driver.find_element(By.ID, "pwPassword")
# 输入密码
password.send_keys(system_dict["password"])

# 等待1s
time.sleep(1)

# 找到 value 为 登录 的按钮
login = driver.find_element(By.XPATH, "//input[@value='登录']")
login.send_keys(Keys.ENTER)

In [ ]:

# 等待1s
time.sleep(1)

print(driver.current_url)

# 输出当前页面的 ID

ids = driver.find_elements(By.XPATH, "//input[@id]")


driver.get("https://icw.buaa.edu.cn/jump.aspx?sysid=wscx")

time.sleep(1)

In [ ]:
print(driver.current_url)

# 查找页面上的所有表格
tables = driver.find_elements(By.TAG_NAME, "table")

print(len(tables))

first_page = []

# 确保页面上至少有两个表格
if len(tables) >= 2:
    # 查找第二个表格的第一行
    second_table = tables[-1]
    # 查找第二个表格中的所有行
    rows = second_table.find_elements(By.TAG_NAME, "tr")

    header_table = tables[-2]
    header = header_table.find_element(By.TAG_NAME, "thead")
    header_row = header.find_element(By.TAG_NAME, "tr")
    # 查找表头行中的所有单元格
    header_cells = header_row.find_elements(By.TAG_NAME, "th")

    # 打印每个表头单元格的文本内容
    header_data = [cell.text for cell in header_cells]
    print("表头信息:", header_data)

    # 遍历每一行并提取信息
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        print(row_data)
        first_page.append(row_data)

driver.quit()

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.utils import formataddr

# 发送邮件
def send_email(data, new_data):

    sender_email = email_dict["email"]
    receiver_email = email_dict["reciver"]
    password = email_dict["password"]

    smtp_server = email_dict["smtp_server"]
    smtp_port = email_dict["smtp_port"]

    message = MIMEMultipart("alternative")
    message["Subject"] = f"{new_data[5]}: {new_data[6]} - {new_data[9]} = {new_data[10]}"
    message["From"] = formataddr(("财务系统更新", sender_email))
    message["To"] = receiver_email

    # Create the email content
    selected_data = data[1:]
    # 转换为 HTML 表格格式
    html_output = "<table border='0'>"
    html_output += "<tr><th>年</th><th>月</th><th>摘要</th><th>金额</th><th>税率</th><th>应发</th></tr>"
    for row in selected_data:
        html_output += "<tr>"

        html_output += f"<td>{row[0]}</td>"
        html_output += f"<td>{row[1]}</td>"
        html_output += f"<td>{row[5]}</td>"
        html_output += f"<td>{row[6]}</td>"
        html_output += f"<td>{row[9]}</td>"
        html_output += f"<td>{row[10]}</td>"

        html_output += "</tr>"
    html_output += "</table>"

    part = MIMEText(html_output, "html")
    message.attach(part)

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Upgrade the connection to a secure encrypted SSL/TLS connection
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        server.quit()

        print("Email sent successfully")
    except Exception as e:
        print(f"Error sending email: {e}")
    pass

In [ ]:
# 将 first page 中的数据写入文件 salary.csv
# 如果文件不存在，将创建文件
# 如果文件存在，对比文件中的数据和 first page 中的数据，如果不一致，将更新文件
# 如果文件存在，对比文件中的数据和 first page 中的数据，如果一致，将不更新文件
# 增强现实新出现的条目

import csv
import os

file_path = 'salary.csv'


def read_csv(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        return list(reader)


def write_csv(file_path, data):
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)


def append_csv(file_path, data):
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)


if os.path.exists(file_path):
    existing_data = read_csv(file_path)
    if existing_data[1:] != first_page:
        # 查找新出现的条目
        new_data = [row for row in first_page if row not in existing_data[1:]]

        if new_data == []:
            new_data = first_page[0]
        else:
            new_data = new_data[0]

        write_csv(file_path, [header_data] + first_page)
        print('File updated with new data.')

        send_email([header_data] + first_page, new_data)

    else:
        print('Data is consistent. No update needed.')
else:
    write_csv(file_path, [header_data] + first_page)
    print('File created and data written.')